In [1]:
%matplotlib inline
from config_calc import *
import grid_tools

from collections import OrderedDict

import cesm_orcas_sci as cesm
import cam
import metpy.calc as mcalc

import matplotlib.pyplot as plt

### read model data set

In [2]:
case = 'bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004a'

tracer_def = cesm.trace_gas_tracers(case)
tracer_3d = [k for k in tracer_def]
varlist = ['Z3','Q','PS','PBLH','T','U','V','U10','PSL','OMEGA']+tracer_3d

dsa = cesm.open_casedata(case,'atm','cam.h0',varlist)
dsa.time

<xarray.DataArray 'time' (time: 3347)>
array(['2007-01-01T00:00:00.000000000', '2007-01-02T00:00:00.000000000',
       '2007-01-03T00:00:00.000000000', ..., '2016-02-27T00:00:00.000000000',
       '2016-02-28T00:00:00.000000000', '2016-02-29T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2007-01-01 2007-01-02 2007-01-03 ...
Attributes:
    long_name:  time
    bounds:     time_bnds

### read flight data
Find the right day in the model data and subset; store a list of obs datasets 

In [37]:
RF = []
MDL = []
co2varname = []
flight_file = ['HIPPO_all_missions_merge_10s_20121129_CO2X.nc','ATom123_2018-01-20_CO2_NOAA.nc']
for f in flight_file:
    rf = xr.open_dataset(os.path.join('data',f))
    
    #-- handle some peculiarities
    co2var = [v for v in rf.variables if 'CO2' in v][0]
    co2 = rf[co2var]
    rf = rf.drop(co2var).rename({co2var:'sample'})
    co2varname.append(co2var)
    rf['sample'] = xr.DataArray(np.arange(0,len(rf.sample)),dims='sample')
    rf['time'] = xr.DataArray([datetime(d[0],d[1],d[2],d[3],d[4],d[5]) for d in rf.Datetime.values],dims='sample')
    rf[co2var] = xr.DataArray(co2.values,dims='sample')
    RF.append(rf)
    
    MDL.append(dsa.sel(time=rf.time[0].astype('M8[D]'),method='nearest'))
dss = xr.concat(MDL,dim='time')

In [38]:
RF[0]

<xarray.Dataset>
Dimensions:    (datedim: 6, sample: 130016)
Coordinates:
  * sample     (sample) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
Dimensions without coordinates: datedim
Data variables:
    Datetime   (sample, datedim) int32 2009 1 9 20 28 40 2009 1 9 20 28 50 ...
    Latitude   (sample) float64 ...
    Longitude  (sample) float64 ...
    Altitude   (sample) float64 ...
    Pressure   (sample) float64 ...
    time       (sample) datetime64[ns] 2009-01-09T20:28:40 ...
    CO2X       (sample) float64 389.4 389.3 389.1 389.0 389.0 389.0 388.8 ...
Attributes:
    history:  Tue Mar 13 16:56:34 2018: ncrename -d Datetime,datedim HIPPO_al...

### compute pressure from hybrid levels

In [4]:
%%time
dss = cam.pres_hybrid(dss,layer_center=True)
varlist = varlist+['Pm']

CPU times: user 649 ms, sys: 4 ms, total: 653 ms
Wall time: 732 ms


In [5]:

dss['theta'] = cam.potential_temperature(pressure=dss.Pm,
                                        temperature=dss['T'])
varlist = varlist+['theta']
dss

<xarray.Dataset>
Dimensions:    (ilev: 57, lat: 192, lev: 56, lon: 288, nbnd: 2, slat: 191, slon: 288, time: 18)
Coordinates:
  * ilev       (ilev) float64 1.651 2.085 2.62 3.276 4.077 5.047 6.217 7.62 ...
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 -86.23 -85.29 -84.35 ...
  * lev        (lev) float64 1.868 2.353 2.948 3.677 4.562 5.632 6.918 8.456 ...
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 6.25 7.5 8.75 10.0 11.25 ...
  * slat       (slat) float64 -89.53 -88.59 -87.64 -86.7 -85.76 -84.82 ...
  * slon       (slon) float64 -0.625 0.625 1.875 3.125 4.375 5.625 6.875 ...
  * time       (time) datetime64[ns] 2016-01-15 2016-01-18 2016-01-21 ...
Dimensions without coordinates: nbnd
Data variables:
    P0         (time) float64 1e+05 1e+05 1e+05 1e+05 1e+05 1e+05 1e+05 ...
    PS         (time, lat, lon) float32 689.061 689.061 689.061 689.061 ...
    Z3         (time, lev, lat, lon) float32 44901.1 44901.1 44901.1 44901.1 ...
    ch4vmr     (time) float64 1.846e-06 

### convert units of tracer consituents

In [6]:
ds = cesm.convert_dataset(dss,case)
ds = ds.compute()
print ds

<xarray.Dataset>
Dimensions:    (ilev: 57, lat: 192, lev: 56, lon: 288, nbnd: 2, slat: 191, slon: 288, time: 18)
Coordinates:
  * ilev       (ilev) float64 1.651 2.085 2.62 3.276 4.077 5.047 6.217 7.62 ...
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 -86.23 -85.29 -84.35 ...
  * lev        (lev) float64 1.868 2.353 2.948 3.677 4.562 5.632 6.918 8.456 ...
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 6.25 7.5 8.75 10.0 11.25 ...
  * slat       (slat) float64 -89.53 -88.59 -87.64 -86.7 -85.76 -84.82 ...
  * slon       (slon) float64 -0.625 0.625 1.875 3.125 4.375 5.625 6.875 ...
  * time       (time) datetime64[ns] 2016-01-15 2016-01-18 2016-01-21 ...
Dimensions without coordinates: nbnd
Data variables:
    P0         (time) float64 1e+05 1e+05 1e+05 1e+05 1e+05 1e+05 1e+05 ...
    PS         (time, lat, lon) float32 689.061 689.061 689.061 689.061 ...
    Z3         (time, lev, lat, lon) float32 44901.1 44901.1 44901.1 44901.1 ...
    ch4vmr     (time) float64 1.846e-06 

### interpolate in xy to get a collection of model columns

In [7]:
%%time
model_colums = []
for i,rf in enumerate(RF):
    var = [ds[v].isel(time=i) for v in varlist]
    model_colums.append(cam.interp_columns_esmf(rf.Longitude,rf.Latitude,ds.lon,ds.lat,*var))
print model_colums[0]

<xarray.Dataset>
Dimensions:   (lev: 56, time: 2723)
Coordinates:
  * time      (time) datetime64[ns] 2016-01-15T12:57:26.000000043 ...
Dimensions without coordinates: lev
Data variables:
    O2_OCN    (time, lev) float64 39.01 39.35 40.18 40.65 40.68 41.16 41.93 ...
    CO2_T09k  (time, lev) float64 -5.06 -5.1 -5.206 -5.272 -5.277 -5.338 ...
    aO2_GKA   (time, lev) float64 14.63 14.79 15.2 15.45 15.47 15.71 16.08 ...
    CO2_C15O  (time, lev) float64 -7.053 -7.112 -7.265 -7.361 -7.369 -7.457 ...
    CO2_LND   (time, lev) float64 1.43 1.446 1.466 1.474 1.473 1.488 1.513 ...
    U10       (time) float64 2.66 2.656 2.651 2.645 2.641 2.639 2.638 2.639 ...
    CO2_T09   (time, lev) float64 -4.792 -4.83 -4.929 -4.991 -4.995 -5.052 ...
    PS        (time) float64 990.8 990.8 990.8 990.7 990.7 990.7 990.7 990.7 ...
    CO2_T09b  (time, lev) float64 -4.681 -4.718 -4.814 -4.874 -4.879 -4.934 ...
    CO2_T09c  (time, lev) float64 -4.748 -4.785 -4.883 -4.944 -4.949 -5.005 ...
    CO2_T09d  (ti

### interpolate within the column to get point of measurement

In [8]:
%%time
rf_model = []
for i,rf in enumerate(RF):
    var = [model_colums[i][v] for v in varlist]
    rf_model.append(cam.interp_within_column(rf.Pressure,model_colums[i].Pm,*var,
                                             model_z_surface=model_colums[i].PS))
        
print rf_model[0]

<xarray.Dataset>
Dimensions:   (time: 2723)
Coordinates:
  * time      (time) datetime64[ns] 2016-01-15T12:57:26.000000043 ...
Data variables:
    O2_OCN    (time) float64 93.15 93.16 93.23 93.27 93.28 92.99 91.75 90.59 ...
    CO2_T09k  (time) float64 -10.39 -10.39 -10.38 -10.37 -10.37 -10.36 ...
    aO2_GKA   (time) float64 47.59 47.6 47.66 47.72 47.76 47.73 47.23 46.44 ...
    CO2_C15O  (time) float64 -14.25 -14.25 -14.25 -14.24 -14.24 -14.23 -14.2 ...
    CO2_LND   (time) float64 1.442 1.435 1.373 1.271 1.23 1.218 1.215 1.306 ...
    U10       (time) float64 2.66 2.656 2.651 2.645 2.641 2.639 2.638 2.639 ...
    CO2_T09   (time) float64 -9.457 -9.457 -9.448 -9.44 -9.434 -9.426 -9.393 ...
    PS        (time) float64 990.8 990.8 990.8 990.7 990.7 990.7 990.7 990.7 ...
    CO2_T09b  (time) float64 -9.086 -9.085 -9.081 -9.077 -9.074 -9.07 -9.055 ...
    CO2_T09c  (time) float64 -9.302 -9.301 -9.292 -9.284 -9.278 -9.271 ...
    CO2_T09d  (time) float64 -9.272 -9.271 -9.264 -9.258 -9.25

In [9]:
copy_flight_var_list = ['Latitude','Longitude','Datetime']
for i,rf in enumerate(RF):
    for v in copy_flight_var_list+[co2varname[i]]:
        rf_model[i][v] = rf[v]

In [10]:
dir_out = os.path.join(os.path.dirname(flight_file[0]),'..','cesm_flight_data')
if not os.path.exists(dir_out):
    call(['mkdir','-p',dir_out])
    
for i,rf in enumerate(rf_model):
    file_out = '.'.join([case,os.path.basename(flight_file[i])])
    file_out = os.path.join(dir_out,file_out)
    print(file_out)
    rf.to_netcdf(file_out,unlimited_dims='time')

/glade/p/work/mclong/orcas/gv_merged_data/../cesm_flight_data/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004a.ORCASrf01.merge10.20170526.nc
/glade/p/work/mclong/orcas/gv_merged_data/../cesm_flight_data/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004a.ORCASrf02.merge10.20170526.nc
/glade/p/work/mclong/orcas/gv_merged_data/../cesm_flight_data/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004a.ORCASrf03.merge10.20170526.nc
/glade/p/work/mclong/orcas/gv_merged_data/../cesm_flight_data/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004a.ORCASrf04.merge10.20170526.nc
/glade/p/work/mclong/orcas/gv_merged_data/../cesm_flight_data/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004a.ORCASrf05.merge10.20170526.nc
/glade/p/work/mclong/orcas/gv_merged_data/../cesm_flight_data/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004a.ORCASrf06.merge10.20170526.nc
/glade/p/work/mclong/orcas/gv_merged_data/../cesm_flight_data/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004a.ORCASrf07.merge10.20170526.nc
/glade/p/work/mclong/orcas/gv_merged_data